<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AIX_7_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#ANSIKTSIGENKÄNNING / FACE RECOGNITION
---
Görs i tre steg:
1.   Lokalisera och 'klipp ut' alla ansikten i bilden
2.   Skapa en vektor (embedding) för varje ansikte
3.   Jämför vektorer för att identifiera personer

OBS: GPU måste användas. Annars kan inte face_recognition importeras.

https://github.com/ageitgey/face_recognition?tab=readme-ov-file

In [ ]:
!pip install face_recognition

In [ ]:
#import cv2
import numpy as np
#import os
import face_recognition
import requests
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [ ]:
# Funktion: Hämta bild från url, lagra som lokal fil
def get_img_to_file(url, path):
  img_data = requests.get(url).content
  with open(path, 'wb') as handler:
      handler.write(img_data)

In [ ]:
# Funktion: Visa upptäckta ansikten
def show_faces(face_locations, image):
  for face_location in face_locations:
      # Print the location of each face in this image
      top, right, bottom, left = face_location
      print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))
      # You can access the actual face itself like this:
      face_image = image[top:bottom, left:right]
      plt.imshow(face_image)
      plt.show()

---
# ENKELT FALL: 2 BILDER - ÄR DET SAMMA PERSON?
---

In [ ]:
get_img_to_file("https://www.download.gubboit.se/musk.JPG", "musk.JPG")
get_img_to_file("https://www.download.gubboit.se/musk2.JPG", "musk2.JPG")

In [ ]:
image = face_recognition.load_image_file("musk.JPG")
plt.imshow(image)
plt.show()
faces = face_recognition.face_locations(image)
show_faces(faces, image)

In [ ]:
image = face_recognition.load_image_file("musk2.JPG")
plt.imshow(image)
plt.show()
faces = face_recognition.face_locations(image)
show_faces(faces, image)

In [ ]:
# Funktion: Visa bild med en 'bounding box' per ansikte
def highlight_faces(image_path, faces):
  plt.figure(figsize=(10, 8))
  # display image
  image = plt.imread(image_path)
  plt.imshow(image)

  ax = plt.gca()

  # for each face, draw a rectangle based on coordinates
  i = 0
  for face in faces:
    top, right, bottom, left = face
    face_border = Rectangle((left, top), right - left, bottom - top,
                          fill=False, color='red')
    plt.text(left, top, str(i), color="red",
             fontdict={"fontsize":20,"fontweight":'bold',"ha":"left", "va":"baseline"})
    ax.add_patch(face_border)
    i = i + 1
  plt.axis('off')
  plt.show()

In [ ]:
image = face_recognition.load_image_file("musk.JPG")
faces = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, faces)
highlight_faces("musk.JPG", faces)

In [ ]:
# Varje ansikte kodas som en vektor med 128 element
print(len(face_encodings[0]))
face_encodings[0]

In [ ]:
image2 = face_recognition.load_image_file("musk2.JPG")
faces2 = face_recognition.face_locations(image2)
face_encodings2 = face_recognition.face_encodings(image2, faces2)
highlight_faces("musk2.JPG", faces2)

In [ ]:
# compare_faces: Kolla om samma person på bilderna
# Ett lägre värde på tolerance ger striktare krav på likhet
face_recognition.compare_faces(face_encodings[0], face_encodings2, tolerance=0.6)

---
#ANSIKTSDETALJER
---
Haka, vänster och höger ögonbryn, näsrygg, nästipp, vänster och höger öga, överläpp, underläpp

In [ ]:
get_img_to_file("https://gfx-bloggar.aftonbladet-cdn.se/wp-content/blogs.dir/518/files/2015/12/brigitte-1024x800.jpg", "star.jpg")

In [ ]:
# Funktion: Visa ansiktsdetaljer
def show_face_landmarks(path):
  # Load the jpg file into a numpy array
  image = face_recognition.load_image_file(path)

  # Find all facial features in all the faces in the image
  face_landmarks_list = face_recognition.face_landmarks(image)

  #image = plt.imread(path)
  plt.figure(figsize=(10, 8))
  plt.imshow(image)

  for face_landmarks in face_landmarks_list:
      # Let's trace out each facial feature in the image with a line!
      for facial_feature in face_landmarks.keys():
        pts = np.vstack(face_landmarks[facial_feature])
        plt.plot(pts[:,0], pts[:,1], color='red', linewidth=2)
  # Show the picture
  plt.axis('off')
  plt.show()

In [ ]:
show_face_landmarks("musk.JPG")

In [ ]:
show_face_landmarks("musk2.JPG")

In [ ]:
show_face_landmarks("star.jpg")

---
### Make up

In [ ]:

from PIL import Image, ImageDraw
# Funktion: Make up
def make_up_landmarks(path):
  # Load the jpg file into a numpy array
  image = face_recognition.load_image_file(path)

  # Find all facial features in all the faces in the image
  face_landmarks_list = face_recognition.face_landmarks(image)

  pil_image = Image.fromarray(image)
  for face_landmarks in face_landmarks_list:
      d = ImageDraw.Draw(pil_image, 'RGBA')

      # Make the eyebrows into a nightmare
      d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
      d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
      d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
      d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

      # Gloss the lips
      d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
      d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
      d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
      d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

      # Sparkle the eyes
      d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
      d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

      # Apply some eyeliner
      d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
      d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)

      plt.imshow(pil_image)
      plt.axis('off')
      plt.show()


In [ ]:
make_up_landmarks("musk.JPG")

In [ ]:
make_up_landmarks("musk2.JPG")

In [ ]:
make_up_landmarks("star.jpg")

---
#CHELSEA - 2 LAGBILDER
---

Vilka spelare från den första bilden finns med på den andra bilden?

In [ ]:
# Spara lagbilderna på fil
get_img_to_file("https://www.download.gubboit.se/chelsea1.jpg", "chelsea1.jpg")
get_img_to_file("https://www.download.gubboit.se/chelsea2.jpg", "chelsea2.jpg")

In [ ]:
# Den första lagbilden
# En större modell behövs - annars missas ett ansikte
image1 = face_recognition.load_image_file("chelsea1.jpg")
faces1 = face_recognition.face_locations(image1, model="cnn")
face_encodings1 = face_recognition.face_encodings(image1, faces1)
highlight_faces("chelsea1.jpg", faces1)

In [ ]:
# Den andra lagbilden
image2 = face_recognition.load_image_file("chelsea2.jpg")
faces2 = face_recognition.face_locations(image2, model="cnn")
face_encodings2 = face_recognition.face_encodings(image2, faces2)
highlight_faces("chelsea2.jpg", faces2)

In [ ]:
# Jämför ansikten på de två lagbilderna => 8 personer finns på båda bilderna
spelare = []
print("Spelare på första bilden som finns med på andra bilden:")
for i in range(len(face_encodings1)):
  # en spelare i bild1 jämförs med spelarna i bild2
  for j in range(len(face_encodings2)):
    result = face_recognition.compare_faces([face_encodings1[i]], face_encodings2[j], tolerance=0.6)
    if (result[0]):
        spelare.append(str(i) + " -> " + str(j))
        break   # vi har hittat en spelare i bild1 som finns i bild2
print(spelare)

---
#JÄMFÖR BILDER PÅ KÄNDISAR
---
##### Vilka bilder föreställer samma person?
##### Kändisar: Ben Affleck, Elton John, Jerry Seinfeld, Madonna, Mindy Kaling

In [ ]:
# Hämta bilderna. 25 bilder - 5 för varje kändis
!rm -f 5celeb.zip*
!wget download.gubboit.se/5celeb.zip

In [ ]:
!unzip -oq 5celeb.zip

In [ ]:
# 25 kändisbilder 1.jpg - 25.jpg.
# 1-5 Ben, 6-10 Elton, 11-15 Jerry, 16-20 Madonna, 21-25 Mindy
!ls 5celeb

In [ ]:
# Skapa vektorer för bilderna och koordinater för ansiktena
model_scores = []
celeb_faces = []
for i in range(1, 26):
  image = face_recognition.load_image_file('5celeb/' + str(i) + '.jpg')
  face = face_recognition.face_locations(image, model="cnn")
  top, right, bottom, left = face[0]
  face_image = image[top:bottom, left:right]
  celeb_faces.append(face_image)
  face_encoding = face_recognition.face_encodings(image, face)
  model_scores.append(face_encoding)

In [ ]:
print(len(model_scores))
model_scores[0]

In [ ]:
# Jämför vektorerna för alla bildpar. Visa bilderna om samma person.
# Testa oxå med tolerance=0.7
differs = []
same = 0
print("Samma person:")
for idx in range(len(model_scores)):
  for idy in range(idx + 1, len(model_scores)):
    result = face_recognition.compare_faces(model_scores[idx][0], model_scores[idy], tolerance=0.6)
    if (result[0]): # samma
      same = same + 1
      # Printing the IDs of faces
      print(idx+1, idy+1)
      #continue
      # Displaying each matched pair of faces
      plt.figure(figsize=(4, 4))
      plt.subplot(1, 2, 1)
      plt.axis('off')
      plt.imshow(celeb_faces[idx])
      plt.subplot(1, 2, 2)
      plt.axis('off')
      plt.imshow(celeb_faces[idy])
      plt.show()
    else: # ej samma
      differs.append(str(idx+1) + ':' + str(idy+1))
print("Av 50 bildpar med samma person hittade vi " + str(same))
print("Olika personer:")
print(differs)

## Kluster

---

Ett annat sätt att bestämma vilka bilder som föreställer samma person är att gruppera bilderna i kluster

In [ ]:
# Varje bild representeras av en vektor
len(model_scores)

In [ ]:
# gör om model_scores till np-arrayer (en matris)
model_scores_np = np.vstack([encoding[0] for encoding in model_scores])
print(model_scores_np.shape)

In [ ]:
# Vi har 5 kändisar - alltså vill vi ha 5 kluster från KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5, init='random', n_init=10, max_iter=300, tol=1e-04, random_state=0)
y_km = km.fit_predict(model_scores_np)
# kluster-index för varje bild
print(y_km)

### Plotta klustren i 2d

In [ ]:
# standardisera - förbättrar resultatet något
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_std = sc.fit_transform(model_scores_np)

In [ ]:
# Minska vektorerna från 128d till 2d med PCA
from sklearn.decomposition import PCA
# init PCA transformer till 2d
pca = PCA(n_components=2)
# dimensionality reduction:
X_pca = pca.fit_transform(X_std)
X_pca.shape

In [ ]:
# Plotta punkterna i 2d
import matplotlib.pyplot as plt
act_part = X_pca[:5]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='x', s=50, label='Ben')
act_part = X_pca[5:10]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='*', edgecolor='green',s=50, label='Elton')
act_part = X_pca[10:15]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='green', marker='o', edgecolor='green',s=50, label='Jerry')
act_part = X_pca[15:20]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='red', marker='o', edgecolor='red',s=50, label='Madonna')
act_part = X_pca[20:25]
plt.scatter(act_part[:, 0], act_part[:, 1],
            c='yellow', marker='o', edgecolor='black',s=50, label='Mindy')
plt.legend(scatterpoints=1)
plt.grid()
plt.title("2d representation (från 128d med PCA)")
plt.tight_layout()
plt.show()

---
#Shaderbooth - mera avancerad än 'Make up'

Prova Shaderbooth på https://shaderbooth.com/

Appen använder webbkameran och 'facial landmarks' för att lägga på olika effekter. Vänta tills appen går igång. Tryck på pilarna för att få olika effekter.